## Modules

In [6]:
import os
import sys
sys.path.append('..')

from datetime import datetime, timedelta
import numpy as np
import pandas as pd
from sqlalchemy import create_engine

from bsoption.nyopchain import NYopchain
Opchain = NYopchain()

from litedata import *
from litedata.litedb import Litedata
from litedata.loadyahoodata import getyahoodata

Litedb = Litedata('yahoo')

### US stock list setting

In [2]:
from configparser import ConfigParser
configchain = ConfigParser()
configchain.read('setting_yahoochain.ini')
chainpath = configchain['paths'].get('chainpath')

configassets = configchain['assets']
etflist = list(configassets.get('etf').split(','))
chipslist = list(configassets.get('chip').split(','))
assetlist = etflist + chipslist

## Gamma Exposure (GEX)

### OHLC of all stocks

In [7]:
dfohlcall = getyahoodata(assetlist)

[*********************100%***********************]  24 of 24 completed


d:\Anaconda\lib\site-packages\IPython\core\interactiveshell.py:2923: PerformanceWarning: indexing past lexsort depth may impact performance.
  return runner(coro)
d:\Anaconda\lib\site-packages\IPython\core\interactiveshell.py:2923: PerformanceWarning: indexing past lexsort depth may impact performance.
  return runner(coro)
d:\Anaconda\lib\site-packages\IPython\core\interactiveshell.py:2923: PerformanceWarning: indexing past lexsort depth may impact performance.
  return runner(coro)
d:\Anaconda\lib\site-packages\IPython\core\interactiveshell.py:2923: PerformanceWarning: indexing past lexsort depth may impact performance.
  return runner(coro)
d:\Anaconda\lib\site-packages\IPython\core\interactiveshell.py:2923: PerformanceWarning: indexing past lexsort depth may impact performance.
  return runner(coro)
d:\Anaconda\lib\site-packages\IPython\core\interactiveshell.py:2923: PerformanceWarning: indexing past lexsort depth may impact performance.
  return runner(coro)
d:\Anaconda\lib\site-p

date
2022-10-03    125.12
2022-10-04    131.67
2022-10-05    132.09
2022-10-06    131.30
2022-10-07    120.76
2022-10-10    116.70
2022-10-11    115.86
2022-10-12    115.00
2022-10-13    119.60
2022-10-14    112.27
2022-10-17    118.88
2022-10-18    119.67
2022-10-19    120.51
2022-10-20    121.94
2022-10-21    124.66
2022-10-24    125.99
2022-10-25    132.61
2022-10-26    128.96
2022-10-27    131.76
2022-10-28    138.34
2022-10-31    134.97
2022-11-01    135.43
2022-11-02    132.19
2022-11-03    134.21
2022-11-04    141.56
Name: NVDA_cl, dtype: float64